In [ ]:
import torch

print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("CUDA NOT AVAILABLE.")


In [ ]:
#CODE BLOCK USED TO COPY THE DATASET FROM GOOGLE DRIVE

import os
import shutil
from tqdm import tqdm

def safe_copy_folder(src_folder, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    errors = []
    total_files = sum(len(files) for _, _, files in os.walk(src_folder))

    #USED FOR TRACKING REAL TIME PROGRESS OF DATA TRANSFER
    with tqdm(total=total_files, desc=f"Copying from {src_folder}") as pbar:
        for root, dirs, files in os.walk(src_folder):
            rel_path = os.path.relpath(root, src_folder)
            target_dir = os.path.join(dest_folder, rel_path)
            os.makedirs(target_dir, exist_ok=True)

            for file in files:
                src_path = os.path.join(root, file)
                dest_path = os.path.join(target_dir, file)
                try:
                    if not os.path.exists(dest_path):
                        shutil.copy2(src_path, dest_path)
                except Exception as e:
                    errors.append((src_path, str(e)))
                pbar.update(1)

    if errors:
        print(f"\n{len(errors)} files failed to copy. Sample error:")
        print(errors[0])
    else:
        print("\n All files copied successfully.")
train_src = r"G:<train_folder_path>"
val_src = r"G:<valid_folder_path"

train_dest = r"Dataset\train"
val_dest = r"Dataset\valid"
#  Not included in submission folder. You can find link for accessing dataset in README.

safe_copy_folder(train_src, train_dest)
safe_copy_folder(val_src, val_dest)

In [ ]:
#CODE BLOCK USED FOR TRAINING THE DATASET

from ultralytics import YOLO

model = YOLO("yolov8l.pt")
model.train(
    data=r"Codes\lunar.yaml",
    epochs=100,
    imgsz=640,
    batch=4,
    name="lunar_exp"
)

In [ ]:
#RESUMING TRAINING FROM LAST.PT

from ultralytics import YOLO

model = YOLO(r"runs\detect\lunar_exp10\weights\last.pt")  #Not included here
model.train(resume=True, epochs=100)


In [ ]:
#DETECTING IMAGES USING MODEL

from ultralytics import YOLO

model = YOLO(r"model\best.pt")
results = model.predict(source= "images", 
                        conf = 0.3,
                        imgsz= 1024,
                        save_txt= True,
                        save= True,
                        project= "train",
                        name= "i_like",
                        exist_ok=True,
                        show_labels=False
                        )


In [ ]:
#CODE TO TRANSFER AND RENAME FILES IN ASKED FORMAT

import os 
import shutil

source_dir = "train/i_like"
label_dir = "train/i_like/labels"

os.makedirs("Results/images", exist_ok=True)
os.makedirs("Results/labels", exist_ok=True)

for file in os.listdir(source_dir):
    if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
        shutil.move(os.path.join(source_dir, file), f"Results/images/{file}")

for file in os.listdir(label_dir):
    if file.endswith(".txt"):
        b = os.path.splitext(file)[0]
        new_name = f"{b}_labels.txt"
        shutil.move(os.path.join(label_dir, file), f"Results/labels/{new_name}")